In [9]:
# Imports & global config
import os
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import statsmodels.api as sm

pd.set_option('display.max_columns', 200)
sns.set(style='whitegrid')
np.random.seed(42)

# DATA_PATH = 'maven_marketing_challenge/marketing_data.csv'
# DICT_PATH = 'maven_marketing_challenge/marketing_data_dictionary.csv'
# EXPORT_PATH = 'maven_marketing_challenge/silver_layer_marketing_data.csv'

DATA_PATH = 'marketing_data.csv'
DICT_PATH = 'marketing_data_dictionary.csv'
EXPORT_PATH = 'silver_layer_marketing_data.csv'

def safe_div(a, b):
    return np.where(b==0, np.nan, a/b)

In [10]:
# Ingest
df = pd.read_csv(DATA_PATH)
data_dict = None
try:
    data_dict = pd.read_csv(DICT_PATH)
except Exception:
    data_dict = None

# Parse likely date columns
for col in df.columns:
    if 'Dt_' in col or 'Date' in col:
        df[col] = pd.to_datetime(df[col], errors='coerce')

print('Shape:', df.shape)
print('Columns:', list(df.columns))
display(df.head(3))

# Null overview
nulls = df.isna().sum().sort_values(ascending=False)
display(nulls[nulls>0])

# Dtype overview
display(df.dtypes)

Shape: (2240, 28)
Columns: ['ID', 'Year_Birth', 'Education', 'Marital_Status', ' Income ', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Response', 'Complain', 'Country']


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
0,1826,1970,Graduation,Divorced,84835.0,0,0,2014-06-16,0,189,104,379,111,189,218,1,4,4,6,1,0,0,0,0,0,1,0,Spain
1,1,1961,Graduation,Single,57091.0,0,0,2014-06-15,0,464,5,64,7,0,37,1,7,3,7,5,0,0,0,0,1,1,0,Canada
2,10476,1958,Graduation,Married,67267.0,0,1,2014-05-13,0,134,11,59,15,2,30,1,3,2,5,2,0,0,0,0,0,0,0,USA


Income     24
dtype: int64

ID                              int64
Year_Birth                      int64
Education                      object
Marital_Status                 object
 Income                       float64
Kidhome                         int64
Teenhome                        int64
Dt_Customer            datetime64[ns]
Recency                         int64
MntWines                        int64
MntFruits                       int64
MntMeatProducts                 int64
MntFishProducts                 int64
MntSweetProducts                int64
MntGoldProds                    int64
NumDealsPurchases               int64
NumWebPurchases                 int64
NumCatalogPurchases             int64
NumStorePurchases               int64
NumWebVisitsMonth               int64
AcceptedCmp3                    int64
AcceptedCmp4                    int64
AcceptedCmp5                    int64
AcceptedCmp1                    int64
AcceptedCmp2                    int64
Response                        int64
Complain    

## Cleaning functions
- Nulls: median for numeric, mode for categorical; dates left as NaT
- Outliers: IQR capping for monetary and count-like variables ($1.5\times IQR$)
- Transformations: $\log(1+x)$ for right-skewed positive variables to stabilize variance

In [11]:
def is_numeric_series(s: pd.Series) -> bool:
    return pd.api.types.is_numeric_dtype(s)

def impute_nulls(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for col in df.columns:
        s = df[col]
        if pd.api.types.is_datetime64_any_dtype(s):
            # Keep NaT; dates often informative when missing
            continue
        if is_numeric_series(s):
            if s.isna().any():
                med = s.median()
                # Median is robust to outliers; preserves distribution center
                df[col] = s.fillna(med)
        else:
            if s.isna().any():
                mode = s.mode(dropna=True)
                fill_val = mode.iloc[0] if len(mode)>0 else 'Unknown'
                df[col] = s.fillna(fill_val)
    return df

def cap_outliers_iqr(df: pd.DataFrame, cols: list, k: float = 1.5) -> pd.DataFrame:
    df = df.copy()
    for col in cols:
        if col not in df.columns or not is_numeric_series(df[col]):
            continue
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - k * iqr
        upper = q3 + k * iqr
        # Capping (winsorizing) preserves sample size; avoids undue influence
        df[col] = np.clip(df[col], lower, upper)
    return df

def add_log_transforms(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    df = df.copy()
    for col in cols:
        if col in df.columns and is_numeric_series(df[col]):
            if (df[col] >= 0).all():
                df[f'log1p_{col}'] = np.log1p(df[col])
    return df

def detect_skewed_positive_cols(df: pd.DataFrame, candidates: list, skew_thr: float = 1.0) -> list:
    skewed = []
    for col in candidates:
        if col in df.columns and is_numeric_series(df[col]):
            s = df[col].dropna()
            if len(s) > 0 and s.min() >= 0:
                sk = stats.skew(s)
                if sk > skew_thr:
                    skewed.append(col)
    return skewed

## Feature engineering
- Age from Year_Birth
- Tenure in months from Dt_Customer
- Total_Spend from product amounts (Mnt*)
- Total_Purchases and channel shares
- Campaign acceptance aggregates
- Web conversion (NumWebPurchases / NumWebVisitsMonth)

In [12]:
def product_amount_cols(df):
    return [c for c in df.columns if c.startswith('Mnt')]

def purchase_count_cols(df):
    return [c for c in df.columns if ('Purchases' in c and c.startswith('Num'))]

def campaign_cols(df):
    # AcceptedCmp1..5 and Response
    acc = [c for c in df.columns if c.startswith('AcceptedCmp')]
    resp = [c for c in df.columns if c == 'Response']
    return acc + resp

def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Age
    if 'Year_Birth' in df.columns:
        current_year = pd.Timestamp.today().year
        df['Age'] = current_year - df['Year_Birth']
        # Cap implausible ages
        df['Age'] = df['Age'].clip(lower=18, upper=100)
    # Tenure in months
    if 'Dt_Customer' in df.columns and pd.api.types.is_datetime64_any_dtype(df['Dt_Customer']):
        df['TenureMonths'] = ((pd.Timestamp.today() - df['Dt_Customer']).dt.days / 30.44).round().astype('Int64')
    # Spend & purchases
    mnt_cols = product_amount_cols(df)
    df['Total_Spend'] = df[mnt_cols].sum(axis=1) if len(mnt_cols)>0 else np.nan
    purch_cols = purchase_count_cols(df)
    df['Total_Purchases'] = df[purch_cols].sum(axis=1) if len(purch_cols)>0 else np.nan
    # Channel shares
    if 'NumWebPurchases' in df.columns:
        df['Share_Web'] = safe_div(df['NumWebPurchases'], df['Total_Purchases'])
    if 'NumCatalogPurchases' in df.columns:
        df['Share_Catalog'] = safe_div(df['NumCatalogPurchases'], df['Total_Purchases'])
    if 'NumStorePurchases' in df.columns:
        df['Share_Store'] = safe_div(df['NumStorePurchases'], df['Total_Purchases'])
    # Web conversion
    if 'NumWebPurchases' in df.columns and 'NumWebVisitsMonth' in df.columns:
        df['Web_Conversion'] = safe_div(df['NumWebPurchases'], df['NumWebVisitsMonth'])
    # Campaign acceptance
    acc_cols = campaign_cols(df)
    if len(acc_cols) > 0:
        df['Campaign_Accepted_Any'] = df[acc_cols].sum(axis=1).clip(upper=1)
        df['Campaign_Accepted_Count'] = df[acc_cols].sum(axis=1)
    return df

## Apply cleaning & feature engineering
- Impute nulls
- Cap outliers for monetary and purchase count columns
- Transform skewed monetary columns

In [13]:
# Make a working copy
work = df.copy()
work = impute_nulls(work)

# Outlier capping (IQR) for spend and count-like columns
mnt_cols = product_amount_cols(work)
num_cols = purchase_count_cols(work)
extra_cols = [c for c in ['Income', 'Recency'] if c in work.columns]
cap_cols = list(set(mnt_cols + num_cols + extra_cols))
work = cap_outliers_iqr(work, cap_cols, k=1.5)

# Log transforms for skewed positive variables
skew_candidates = mnt_cols + [c for c in ['Income'] if c in work.columns]
skewed = detect_skewed_positive_cols(work, skew_candidates, skew_thr=1.0)
work = add_log_transforms(work, skewed)

# Engineer features
work = engineer_features(work)
display(work.head(3))

# Basic sanity checks
assert work.shape[0] == df.shape[0], 'Row count changed unexpectedly.'
print('Engineered columns added:', [c for c in work.columns if c not in df.columns][:10], '...')

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country,log1p_MntWines,log1p_MntFruits,log1p_MntMeatProducts,log1p_MntFishProducts,log1p_MntSweetProducts,log1p_MntGoldProds,Age,TenureMonths,Total_Spend,Total_Purchases,Share_Web,Share_Catalog,Share_Store,Web_Conversion,Campaign_Accepted_Any,Campaign_Accepted_Count
0,1826,1970,Graduation,Divorced,84835.0,0,0,2014-06-16,0,189,81,379,111.0,81,126.5,1,4,4,6,1,0,0,0,0,0,1,0,Spain,5.247024,4.406719,5.940171,4.718499,4.406719,4.848116,55,138,967.5,15,0.266667,0.266667,0.400000,4.0,1,1
1,1,1961,Graduation,Single,57091.0,0,0,2014-06-15,0,464,5,64,7.0,0,37.0,1,7,3,7,5,0,0,0,0,1,1,0,Canada,6.142037,1.791759,4.174387,2.079442,0.000000,3.637586,64,138,577.0,18,0.388889,0.166667,0.388889,1.4,1,2
2,10476,1958,Graduation,Married,67267.0,0,1,2014-05-13,0,134,11,59,15.0,2,30.0,1,3,2,5,2,0,0,0,0,0,0,0,USA,4.905275,2.484907,4.094345,2.772589,1.098612,3.433987,67,139,251.0,11,0.272727,0.181818,0.454545,1.5,0,0


Engineered columns added: ['log1p_MntWines', 'log1p_MntFruits', 'log1p_MntMeatProducts', 'log1p_MntFishProducts', 'log1p_MntSweetProducts', 'log1p_MntGoldProds', 'Age', 'TenureMonths', 'Total_Spend', 'Total_Purchases'] ...


## Q1: Factors related to number of web purchases
Approach:
- Pearson correlations
- OLS with standardized predictors
Correlation formula: $$\rho_{X,Y} = \frac{n\sum XY - (\sum X)(\sum Y)}{\sqrt{\left[n\sum X^2 - (\sum X)^2\right]\left[n\sum Y^2 - (\sum Y)^2\right]}}$$

In [14]:
targets = ['NumWebPurchases']
predictors_pref = [
    'NumWebVisitsMonth','Income','Total_Spend','Age','Recency',
    'NumCatalogPurchases','NumStorePurchases','TenureMonths','Campaign_Accepted_Count'
]
predictors = [c for c in predictors_pref if c in work.columns]
y = work[targets[0]] if targets[0] in work.columns else None
X = work[predictors].copy() if len(predictors)>0 else None

if y is not None and X is not None:
    # Correlations
    corr = pd.Series({c: np.corrcoef(work[c], y)[0,1] for c in predictors})
    display(corr.sort_values(ascending=False))
    
    # OLS with standardized predictors
    X_std = (X - X.mean()) / X.std(ddof=0)
    X_std = sm.add_constant(X_std)
    # Ensure numeric dtypes to avoid object/pd.NA issues
    X_std = X_std.apply(pd.to_numeric, errors='coerce').astype(float)
    y_num = pd.to_numeric(y, errors='coerce').astype(float)
    model = sm.OLS(y_num, X_std, missing='drop').fit()
    print(model.summary())
else:
    print('Required columns for web purchases analysis not found.')

Total_Spend                0.576686
NumStorePurchases          0.543317
NumCatalogPurchases        0.448309
Campaign_Accepted_Count    0.220177
TenureMonths               0.205884
Age                        0.154899
Recency                   -0.004153
NumWebVisitsMonth         -0.041648
dtype: float64

                            OLS Regression Results                            
Dep. Variable:        NumWebPurchases   R-squared:                       0.472
Model:                            OLS   Adj. R-squared:                  0.470
Method:                 Least Squares   F-statistic:                     248.9
Date:                Wed, 24 Dec 2025   Prob (F-statistic):          2.39e-302
Time:                        16:22:22   Log-Likelihood:                -4644.3
No. Observations:                2240   AIC:                             9307.
Df Residuals:                    2231   BIC:                             9358.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

## Q2: Which marketing campaign is most successful?
Measured by acceptance (mean of AcceptedCmpX or Response).

In [15]:
acc_cols = [c for c in work.columns if c.startswith('AcceptedCmp')] + [c for c in work.columns if c=='Response']
if len(acc_cols)>0:
    rates = work[acc_cols].mean().sort_values(ascending=False)
    display(rates.to_frame('AcceptanceRate'))
    print('Top campaign:', rates.index[0])
else:
    print('No campaign columns found.')

,AcceptanceRate
Response,0.149107
AcceptedCmp4,0.074554
AcceptedCmp3,0.072768
AcceptedCmp5,0.072768
AcceptedCmp1,0.064286
AcceptedCmp2,0.013393


Top campaign: Response


## Q3: What does the average customer look like?

In [16]:
profile_cols = [c for c in ['Age','Income','TenureMonths','Total_Spend','Total_Purchases','Share_Web','Share_Catalog','Share_Store'] if c in work.columns]
display(work[profile_cols].describe().loc[['mean','50%']].rename(index={'50%':'median'}))

cat_cols = [c for c in ['Education','Marital_Status','Kidhome','Teenhome'] if c in work.columns]
for c in cat_cols:
    display(work[c].value_counts(normalize=True).mul(100).round(1).rename('pct'))

,Age,TenureMonths,Total_Spend,Total_Purchases,Share_Web,Share_Catalog,Share_Store
mean,56.157143,149.458036,568.206696,14.690625,0.268235,0.142212,0.411178
median,55.000000,150.0,395.000000,15.000000,0.260870,0.125000,0.411765


Education
Graduation    50.3
PhD           21.7
Master        16.5
2n Cycle       9.1
Basic          2.4
Name: pct, dtype: float64

Marital_Status
Married     38.6
Together    25.9
Single      21.4
Divorced    10.4
Widow        3.4
Alone        0.1
YOLO         0.1
Absurd       0.1
Name: pct, dtype: float64

Kidhome
0    57.7
1    40.1
2     2.1
Name: pct, dtype: float64

Teenhome
0    51.7
1    46.0
2     2.3
Name: pct, dtype: float64

## Q4: Which products are performing best?
Rank by total spend and average spend per customer.

In [17]:
mnt_cols = [c for c in work.columns if c.startswith('Mnt')]
if len(mnt_cols)>0:
    totals = work[mnt_cols].sum().sort_values(ascending=False)
    avgs = work[mnt_cols].mean().sort_values(ascending=False)
    print('Total spend by product:')
    display(totals.to_frame('TotalSpend'))
    print('Average spend by product:')
    display(avgs.to_frame('AvgSpend'))
else:
    print('No product amount columns found.')

Total spend by product:


,TotalSpend
MntWines,676733.0
MntMeatProducts,337956.0
MntGoldProds,88452.5
MntFishProducts,71939.5
MntSweetProducts,49141.0
MntFruits,48561.0


Average spend by product:


,AvgSpend
MntWines,302.112946
MntMeatProducts,150.873214
MntGoldProds,39.487723
MntFishProducts,32.115848
MntSweetProducts,21.937946
MntFruits,21.679018


## Q5: Which channels are underperforming?
Approximate conversion rates:
- Web: NumWebPurchases / NumWebVisitsMonth
- Store & Catalog: relative purchase shares (as proxy if no visit/contact counts)

In [18]:
metrics = {}
if 'NumWebPurchases' in work.columns and 'NumWebVisitsMonth' in work.columns:
    metrics['Web_Conversion_Rate'] = np.nanmean(work['Web_Conversion'])
if 'NumStorePurchases' in work.columns and 'Total_Purchases' in work.columns:
    metrics['Store_Share'] = np.nanmean(work['Share_Store'])
if 'NumCatalogPurchases' in work.columns and 'Total_Purchases' in work.columns:
    metrics['Catalog_Share'] = np.nanmean(work['Share_Catalog'])
print('Channel metrics:')
display(pd.Series(metrics))

# Identify underperforming as lowest conversion/share
if len(metrics)>0:
    under = pd.Series(metrics).sort_values(ascending=True)
    print('Underperforming channel (lowest metric):', under.index[0])
else:
    print('Not enough columns to compute channel performance.')

Channel metrics:


Web_Conversion_Rate    1.072769
Store_Share            0.411178
Catalog_Share          0.142212
dtype: float64

Underperforming channel (lowest metric): Catalog_Share


## Export silver layer
Save cleaned and enriched dataset for Power BI.

In [19]:
work.to_csv(EXPORT_PATH, index=False)
print('Exported:', EXPORT_PATH, 'shape:', work.shape)

Exported: silver_layer_marketing_data.csv shape: (2240, 44)
